[< Back to homepage](/notebooks/Home.ipynb)

# Time Based Analysis

Time-based analysis relies heavily on date functions and ranges, which are documented [here](https://developer.salesforce.com/docs/atlas.en-us.bi_dev_guide_eql.meta/bi_dev_guide_eql/bi_eql_functions_date.htm). Some common examples are detailed below.

In [22]:
%run saql_init.py

## MTD (Month To Date)
Calculate the total value of the opportunities expected to close from the beginning of the month through today.


In [23]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current month".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,21078458.0


The relative date range filters for the beginning of the month through today. As the month or day changes, these keywords will refer to the current month and day.

## YTD (Year To Date)

Calculate the total value of the opportunities expected to close from the beginning of the year through today.

In [24]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current year".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,178020414.0


## QTD (Quarter To Date)

Calculate the total value of the opportunities expected to close from the beginning of the quarter through today.

In [25]:
%%saql
q = load "opportunities";
q = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current day"];
q = group q by all;
q = foreach q generate sum('Amount') as 'sum_Amount';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_Amount
0,21078458.0


## QoQ (Quarter over Quarter)
Calculate the percent change in total value of the opportunities expected to close for the current quarter compared to last quarter.

In [26]:
%%saql
q = load "opportunities";
curr = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["current quarter".."current quarter"];
prev = filter q by date('CloseDate_Year', 'CloseDate_Month', 'CloseDate_Day') in ["1 quarter ago".."1 quarter ago"];
QoQ = group curr by all, prev by all;
QoQ = foreach QoQ generate (sum(curr['Amount'])/sum(prev['Amount'])) - 1 as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,sum_QoQ
0,-0.152189


** Query Explanation:**

After loading in the opportunity dataset, we split it into two streams. One stream, curr, gets data for the current quarter using the date range `["current quarter".."current quarter"]` which filters for data ranging from the start of the current quarter to the end of the current quarter. The other stream, prev, gets data for the previous quarter. We then bring the two streams back together using a co-group. Since we are trying to calculate the overall QoQ number, we can group by alland then perform the QoQ calculation.

### Windowing Alternative

Let us see how we can do the same QoQ using Windowing.

In [27]:
%%saql
q = load "opportunities";
q = group q by ('CloseDate_Year', 'CloseDate_Quarter');
q = foreach q generate 'CloseDate_Year' + "~~~" + 'CloseDate_Quarter' as 'CloseDate_Year~~~CloseDate_Quarter',
    ((sum('Amount') / sum(sum('Amount')) over([-1 .. -1] partition by all order by ('CloseDate_Year', 'CloseDate_Quarter'))) - 1) as 'sum_QoQ';

User "kim@waverocks.de" running query on https://na30.salesforce.com


,CloseDate_Year~~~CloseDate_Quarter,sum_QoQ
0,2015~~~1,NaN
1,2015~~~2,0.281156
2,2015~~~3,0.694430
3,2015~~~4,0.379414
4,2016~~~1,-0.089058
5,2016~~~2,-0.152189
6,2016~~~3,-0.405879
7,2016~~~4,-0.786054
